In [1]:
import pandas as pd
from config import weather_api_key
import datetime
import time
import requests
from pprint import pprint

In [2]:
# Reading 2015 csv file
data_15_df = pd.read_csv('clean_data/2015_weather_cancels_only.csv')
data_15_df.tail()

,Date,Origin,Destination,Expected Departure Time,Actual Departure Time,Departure Delay,Expected Arrival Time,Distance
48846,2015-12-31,IAH,BRO,900,0.0,0.0,1020,308.0
48847,2015-12-31,VLD,ATL,615,0.0,0.0,718,208.0
48848,2015-12-31,CAE,ATL,730,0.0,0.0,843,192.0
48849,2015-12-31,ABE,ATL,600,0.0,0.0,813,692.0
48850,2015-12-31,ABE,ORD,722,0.0,0.0,842,655.0


In [3]:
# Reading gloabal airport data csv
status_df = pd.read_csv('GlobalAirportDatabase/GlobalAirportDatabaseClean.csv')
status_df =status_df.loc[status_df["Country"]== "USA"]
status_df.head()

,ICAO Code,IATA Code,Airport Name,City/Town,Country,Latitude Decimal Degrees,Longitude Decimal Degrees
3380,KABI,ABI,ABILENE RGNL,ABILENE,USA,32.411,-99.682
3381,KABQ,ABQ,NaN,ALBUQUERQUE,USA,0.000,0.000
3382,KACK,ACK,NANTUCKET MEM,NANTUCKET,USA,41.253,-70.060
3383,KACT,ACT,WACO RGNL,WACO,USA,31.611,-97.230
3384,KACY,ACY,ATLANTIC CITY INTERNATIONAL,ATLANTIC CITY,USA,39.458,-74.577


In [4]:
# Mergeing 2015 csv and global airport data to get the longitude and latitude for the origin
combined_15_origin_df = pd.merge(data_15_df,status_df,left_on="Origin",right_on="IATA Code")
combined_15_origin_df.head()

,Date,Origin,Destination,Expected Departure Time,Actual Departure Time,Departure Delay,Expected Arrival Time,Distance,ICAO Code,IATA Code,Airport Name,City/Town,Country,Latitude Decimal Degrees,Longitude Decimal Degrees
0,2015-01-01,ATL,DFW,1124,0.0,0.0,1252,731.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
1,2015-01-01,ATL,DFW,625,0.0,0.0,755,731.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
2,2015-01-02,ATL,MCI,1910,0.0,0.0,2029,692.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
3,2015-01-02,ATL,MCI,2155,0.0,0.0,2313,692.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
4,2015-01-02,ATL,BTR,1400,0.0,0.0,1438,448.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427


In [5]:
# renaming the latitude and longitudes to origin lat and origin lon
combined_15_origin_df=combined_15_origin_df.rename(columns={"Latitude Decimal Degrees":"Origin Latitude",
                                                          "Longitude Decimal Degrees":"Origin Longitude",})
combined_15_origin_df.head()

,Date,Origin,Destination,Expected Departure Time,Actual Departure Time,Departure Delay,Expected Arrival Time,Distance,ICAO Code,IATA Code,Airport Name,City/Town,Country,Origin Latitude,Origin Longitude
0,2015-01-01,ATL,DFW,1124,0.0,0.0,1252,731.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
1,2015-01-01,ATL,DFW,625,0.0,0.0,755,731.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
2,2015-01-02,ATL,MCI,1910,0.0,0.0,2029,692.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
3,2015-01-02,ATL,MCI,2155,0.0,0.0,2313,692.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427
4,2015-01-02,ATL,BTR,1400,0.0,0.0,1438,448.0,KATL,ATL,THE WILLIAM B HARTSFIELD ATLANTA INTERNATIONAL,ATLANTA,USA,33.64,-84.427


In [6]:
# merging 2015 csv and global airport date to get the destination latitude and destination longitude
combined_15_dest_df = pd.merge(combined_15_origin_df,status_df,left_on="Destination",right_on="IATA Code")
combined_15_dest_df=combined_15_dest_df.rename(columns={"Latitude Decimal Degrees":"Destination Latitude",
                                                          "Longitude Decimal Degrees":"Destination Longitude",})
combined_15_dest_df.head()

,Date,Origin,Destination,Expected Departure Time,Actual Departure Time,Departure Delay,Expected Arrival Time,Distance,ICAO Code_x,IATA Code_x,...,Country_x,Origin Latitude,Origin Longitude,ICAO Code_y,IATA Code_y,Airport Name_y,City/Town_y,Country_y,Destination Latitude,Destination Longitude
0,2015-01-01,ATL,DFW,1124,0.0,0.0,1252,731.0,KATL,ATL,...,USA,33.64,-84.427,KDFW,DFW,DALLAS FORT WORTH INTERNATIONAL,DALLAS-FORT WORTH,USA,32.896,-97.037
1,2015-01-01,ATL,DFW,625,0.0,0.0,755,731.0,KATL,ATL,...,USA,33.64,-84.427,KDFW,DFW,DALLAS FORT WORTH INTERNATIONAL,DALLAS-FORT WORTH,USA,32.896,-97.037
2,2015-02-23,ATL,DFW,1849,0.0,0.0,2020,731.0,KATL,ATL,...,USA,33.64,-84.427,KDFW,DFW,DALLAS FORT WORTH INTERNATIONAL,DALLAS-FORT WORTH,USA,32.896,-97.037
3,2015-02-23,ATL,DFW,1510,0.0,0.0,1638,731.0,KATL,ATL,...,USA,33.64,-84.427,KDFW,DFW,DALLAS FORT WORTH INTERNATIONAL,DALLAS-FORT WORTH,USA,32.896,-97.037
4,2015-02-23,ATL,DFW,1755,0.0,0.0,1928,731.0,KATL,ATL,...,USA,33.64,-84.427,KDFW,DFW,DALLAS FORT WORTH INTERNATIONAL,DALLAS-FORT WORTH,USA,32.896,-97.037


In [7]:
# reducing the dataframe
final_15_df=combined_15_dest_df[["Date","Origin","Destination","Expected Departure Time","Expected Arrival Time","Distance",
                             "Origin Latitude","Origin Longitude","Destination Latitude","Destination Longitude"]]
final_15_df.head()

,Date,Origin,Destination,Expected Departure Time,Expected Arrival Time,Distance,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,2015-01-01,ATL,DFW,1124,1252,731.0,33.64,-84.427,32.896,-97.037
1,2015-01-01,ATL,DFW,625,755,731.0,33.64,-84.427,32.896,-97.037
2,2015-02-23,ATL,DFW,1849,2020,731.0,33.64,-84.427,32.896,-97.037
3,2015-02-23,ATL,DFW,1510,1638,731.0,33.64,-84.427,32.896,-97.037
4,2015-02-23,ATL,DFW,1755,1928,731.0,33.64,-84.427,32.896,-97.037


In [8]:
# getting sample from the reduced dataframe
sample_15_df = final_15_df.sample(n=4)
sample_15_df["Origin Wind Speed"]=""
sample_15_df["Destination Wind Speed"]= ""
sample_15_df["Origin Temp"] = ""
sample_15_df["Dest Temp"] = ""
sample_15_df["Origin Wind gust"]=""
sample_15_df["Dest Wind gust"]=""
sample_15_df["Origin Visibility"]=""
sample_15_df["Dest Visibility"] =""
sample_15_df["Origin Condition"] =""
sample_15_df["Dest Condition"] =""
sample_15_df["Origin Precip"] =""
sample_15_df["Dest Precip"] =""
sample_15_df["Origin Precip Prob"] =""
sample_15_df["Dest Precip Prob"] =""
sample_15_df

,Date,Origin,Destination,Expected Departure Time,Expected Arrival Time,Distance,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,...,Origin Wind gust,Dest Wind gust,Origin Visibility,Dest Visibility,Origin Condition,Dest Condition,Origin Precip,Dest Precip,Origin Precip Prob,Dest Precip Prob
10043,2015-02-04,GRR,ORD,829,830,137.0,42.881,-85.523,41.979,-87.904,...,,,,,,,,,,
19011,2015-01-26,JFK,BOS,2000,2122,187.0,40.640,-73.779,42.364,-71.005,...,,,,,,,,,,
3576,2015-03-04,EWR,DFW,1448,1755,1372.0,40.692,-74.169,32.896,-97.037,...,,,,,,,,,,
24126,2015-12-15,TUS,DEN,710,900,639.0,32.116,-110.941,39.858,-104.667,...,,,,,,,,,,


In [9]:
# loop through the sample dataframe
for index,row in sample_15_df.iterrows():
    
    #----------Origin latitude and Longitude
    lat=row["Origin Latitude"]
    lng=row["Origin Longitude"]
    datetime= row["Date"]
    
   # convert the departure time to string to find the substring
    hour_time= str(row["Expected Departure Time"])
    
   # check this condition to extract the exact hour
    if(row["Expected Departure Time"]>999):
        hour_time_sub = hour_time[0:2]
    else:
        hour_time_sub = hour_time[0:1]
    hour_time_int = int(hour_time_sub)
    print(hour_time_int)

    # Build URL
    base_url ="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    query = (f"{lat},{lng}/{datetime}/?key={weather_api_key}")
    new_url= base_url + query
#     print(new_url)
#     get the response
    response = requests.get(new_url).json()
    
    # Use try and except to skip the missing data
    try:
        sample_15_df.loc[index,"Origin Wind Speed"]=response['days'][0]['hours'][hour_time_int]['windspeed'] 
        sample_15_df.loc[index,"Origin Temp"] = response['days'][0]['hours'][hour_time_int]['temp']
        sample_15_df.loc[index,"Origin Wind gust"] = response['days'][0]['hours'][hour_time_int]['windgust']
        sample_15_df.loc[index,"Origin Visibility"] = response['days'][0]['hours'][hour_time_int]['visibility']
        sample_15_df.loc[index,"Origin Condition"] = response['days'][0]['hours'][hour_time_int]['conditions']
        sample_15_df.loc[index,"Origin Precip"] = response['days'][0]['hours'][hour_time_int]['precip']
        sample_15_df.loc[index,"Origin Precip Prob"] = response['days'][0]['hours'][hour_time_int]['precipprob']
    except (KeyError, IndexError):
        print("Data not found... skipping.")
        
    #----------Destination latitude and Longitude
    des_lat=row["Destination Latitude"]
    des_lng=row["Destination Longitude"]
    hour_time= str(row["Expected Arrival Time"])
    # check this condition to extract the exact hour
    if(row["Expected Arrival Time"]>999):
        hour_time_sub = hour_time[0:2]
    else:
        hour_time_sub = hour_time[0:1]
    hour_time_int = int(hour_time_sub)
    print(hour_time_int)

    # Build URL
    des_query = (f"{des_lat},{des_lng}/{datetime}/?key={weather_api_key}")
    new_url= base_url + des_query
#     print(new_url)
    # get the response
    response = requests.get(new_url).json()
    
    # Use try and except to skip the missing data
    try:
        sample_15_df.loc[index,"Destination Wind Speed"]= response['days'][0]['hours'][hour_time_int]['windspeed']   
        sample_15_df.loc[index,"Dest Temp"] = response['days'][0]['hours'][hour_time_int]['temp']
        sample_15_df.loc[index,"Dest Wind gust"] = response['days'][0]['hours'][hour_time_int]['windgust']
        sample_15_df.loc[index,"Dest Visibility"] = response['days'][0]['hours'][hour_time_int]['visibility']
        sample_15_df.loc[index,"Dest Condition"] = response['days'][0]['hours'][hour_time_int]['conditions']
        sample_15_df.loc[index,"Dest Precip"] = response['days'][0]['hours'][hour_time_int]['precip']
        sample_15_df.loc[index,"Dest Precip Prob"] = response['days'][0]['hours'][hour_time_int]['precipprob']
    except (KeyError, IndexError):
        print("Data not found... skipping.")

#     pprint(response)


8
8
20
21
14
17
7
9


In [10]:
# 2015 sample with response data
sample_15_df

,Date,Origin,Destination,Expected Departure Time,Expected Arrival Time,Distance,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,...,Origin Wind gust,Dest Wind gust,Origin Visibility,Dest Visibility,Origin Condition,Dest Condition,Origin Precip,Dest Precip,Origin Precip Prob,Dest Precip Prob
10043,2015-02-04,GRR,ORD,829,830,137.0,42.881,-85.523,41.979,-87.904,...,None,None,6.8,6.8,Overcast,Overcast,0,0,0,0
19011,2015-01-26,JFK,BOS,2000,2122,187.0,40.640,-73.779,42.364,-71.005,...,35.4,37.8,0.6,1.2,"Snow, Overcast","Snow, Overcast",0.04,0.04,100,100
3576,2015-03-04,EWR,DFW,1448,1755,1372.0,40.692,-74.169,32.896,-97.037,...,None,28.9,6.2,1.9,Overcast,"Snow, Rain, Overcast",0,0.02,0,100
24126,2015-12-15,TUS,DEN,710,900,639.0,32.116,-110.941,39.858,-104.667,...,None,29.8,9.9,0,Partially cloudy,Overcast,0,0,0,0


In [11]:
sample_15_origin = sample_15_df[["Date","Origin","Expected Departure Time","Origin Wind Speed","Origin Temp","Origin Latitude","Origin Longitude","Origin Wind gust",
                                 "Origin Visibility","Origin Condition","Origin Precip","Origin Precip Prob"]]
sample_15_origin 

,Date,Origin,Expected Departure Time,Origin Wind Speed,Origin Temp,Origin Latitude,Origin Longitude,Origin Wind gust,Origin Visibility,Origin Condition,Origin Precip,Origin Precip Prob
10043,2015-02-04,GRR,829,8.1,21.8,42.881,-85.523,None,6.8,Overcast,0,0
19011,2015-01-26,JFK,2000,23,27,40.640,-73.779,35.4,0.6,"Snow, Overcast",0.04,100
3576,2015-03-04,EWR,1448,8,42.2,40.692,-74.169,None,6.2,Overcast,0,0
24126,2015-12-15,TUS,710,4.8,30.9,32.116,-110.941,None,9.9,Partially cloudy,0,0


In [12]:
sample_15_dest = sample_15_df[["Date","Destination","Expected Arrival Time","Destination Wind Speed","Dest Temp","Destination Latitude","Destination Longitude","Dest Wind gust",
                               "Dest Visibility","Dest Condition","Dest Precip","Dest Precip Prob"]]
sample_15_dest

,Date,Destination,Expected Arrival Time,Destination Wind Speed,Dest Temp,Destination Latitude,Destination Longitude,Dest Wind gust,Dest Visibility,Dest Condition,Dest Precip,Dest Precip Prob
10043,2015-02-04,ORD,830,15,18,41.979,-87.904,None,6.8,Overcast,0,0
19011,2015-01-26,BOS,2122,23,26.9,42.364,-71.005,37.8,1.2,"Snow, Overcast",0.04,100
3576,2015-03-04,DFW,1755,23,31.9,32.896,-97.037,28.9,1.9,"Snow, Rain, Overcast",0.02,100
24126,2015-12-15,DEN,900,27.7,17.5,39.858,-104.667,29.8,0,Overcast,0,0


In [ ]:
# data_16_df = pd.read_csv('clean_data/2016_weather_cancels_only.csv')
# data_16_df

In [ ]:
combined_16_df = pd.merge(data_16_df,status_df,left_on="Origin",right_on="IATA Code")
combined_16_df

In [ ]:
# rows = combined_16_df.sample(n=1000)
# rows